In [4]:
from modules.utils import *
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### 파일 분리

In [5]:

def split_pdf(filepath, batch_size=5):
    """
    입력 PDF를 분할 PDF 파일로 분할
    """
    # PDF 파일 열기
    basename = os.path.splitext(filepath)[0]
    filename = os.path.splitext(os.path.basename(basename))[0]
    os.makedirs(basename, exist_ok=True)
    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)

    ret = []
    # PDF 분할
    for start_page in range(0, num_pages, batch_size):
        end_page = min(start_page + batch_size, num_pages) - 1
        # 분할된 PDF 저장
        # /folder/example.pdf = > ['/folder/example','.pdf']
        output_file = f"{basename}\\{filename}_{start_page:04d}_{end_page:04d}.pdf"
        print(f"분할 PDF 생성: {output_file}")
        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    # 입력 PDF 파일 닫기
    input_pdf.close()

    return ret, basename

In [6]:
filepath = ".cache\\files\\seq2seq.pdf"
file_paths,basename = split_pdf(filepath)

print(file_paths,basename)

분할 PDF 생성: .cache\files\seq2seq\seq2seq_0000_0004.pdf
분할 PDF 생성: .cache\files\seq2seq\seq2seq_0005_0008.pdf
['.cache\\files\\seq2seq\\seq2seq_0000_0004.pdf', '.cache\\files\\seq2seq\\seq2seq_0005_0008.pdf'] .cache\files\seq2seq


### upstage에서 데이터 파싱

In [8]:
analysis_file_path = upstage_layout_analysis(file_paths)

In [ ]:
# analysis_file_path = ['.cache\\files\\seq2seq\\seq2seq_0000_0004.json', '.cache\\files\\seq2seq\\seq2seq_0005_0008.json']
analysis_file_path

### 각 파일 분리하여 저장
- page 번호 조정
- id 조정
- html id tag 조정

In [9]:
def reOrder_id(analysis_file_path):
    element_content = []
    page_range = 5
    last_number = 0
    for i, path in enumerate(sorted(analysis_file_path)):
        with open(path, "r", encoding="utf-8") as file:
            # 파일 내용을 파이썬 객체로 변환
            data = json.load(file)  # 보통 JSON 배열이면 list 타입이 됨
            change_page = page_range * i
            for j, element in enumerate(data["elements"]):
                element["id"] = last_number + j
                html_content = element["content"]["html"]
                soup = BeautifulSoup(html_content, "html.parser")
                tag = soup.find(attrs={"id": True})
                tag["id"] = last_number + j
                element["content"]["html"] = str(tag)
                element["page"] = change_page + element["page"]
            last_number = len(data["elements"])
            element_content.extend(data["elements"])
    return element_content

In [10]:
element_content = reOrder_id(analysis_file_path)
# print(element_content)
# with open(f"{output_file_basename}_merged.json", "w", encoding="utf-8") as json_file:
#     json.dump(
#         element_content,
#         json_file,
#     )

### 분리한 파일에서 이미지 가져와서 저장

In [11]:
import json
import base64
import os
from bs4 import BeautifulSoup

# input_json_path = f"{output_file_basename}_merged.json"  # JSON 파일 경로
# output_dir = output_file_basename  # 이미지 저장 폴더
# print(input_json_path)
# # 저장 폴더가 없으면 생성
# os.makedirs(output_dir, exist_ok=True)
# with open(input_json_path, "r", encoding="utf-8") as f:
#     data = json.load(f)  # JSON 배열 데이터 로드

def extract_image(element_content):
    arr = []
    for idx, item in enumerate(element_content):
        _dict = dict()
        if "base64_encoding" in item:
            _dict = {"base64_encoding": item["base64_encoding"]}
            html_str = item["content"]["html"]
            soup = BeautifulSoup(html_str, "html.parser")
            img_tags = soup.find_all("img")
            if img_tags:
                base64_str = item["base64_encoding"]
                # base64 문자열 디코딩
                image_data = base64.b64decode(base64_str)
                image_path = os.path.join(f"{basename}\\image_{idx}.png")
                for img_tag in soup.find_all("img"):
                    # 기존 속성 모두 제거
                    img_tag.attrs.clear()
                    # 원하는 텍스트를 src 속성에 넣기
                    img_tag["src"] = image_path
                # 이미지 파일 저장
                _dict["content_text"] = str(img_tag)
                with open(f"{image_path}", "wb") as img_file:
                    img_file.write(image_data)
                print(f"Saved image: {image_path}")
            else:
                _dict["content_text"] = item["content"]["html"]
                print("이미지 태그가 없습니다.")
        else:
            _dict["content_text"] = item["content"]["html"]
            print("base64_encoding params 없습니다.")

        _dict["metadata"] = {"id": item["id"], "page": item["page"]}
        arr.append(_dict)
    return arr


In [12]:
_element_content = extract_image(element_content)
# print(_element_content)

base64_encoding params 없습니다.
이미지 태그가 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
Saved image: .cache\files\seq2seq\image_12.png
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_encoding params 없습니다.
base64_enco

### content_text 에서 html 태그 가져와 tag에 img 존재하는 경우 모아서 summary 진행 

In [13]:
import json
import base64
import os
from bs4 import BeautifulSoup


def get_system_prompt():
    return """You are an expert in extracting useful information from IMAGE.
    In particular, you specialize in analyzing papers.
    With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval."""


def get_user_image_template(previous_context, next_context, image_paths, id):
    return f"""
        Please consider the following text context—both the preceding paragraph and the following paragraph—along with the included image. 
        Based on this context, provide a simple and clear explanation of the image, suitable for someone with no prior background knowledge. 
        Use easy-to-understand language and focus on the main ideas that the image conveys in relation to the text.

        [Preceding paragraph text]
        {previous_context}
        [Following paragraph text]
        {next_context}


        ###
        Output Format:
        <image>
        <title>
        <summary>
        <entities> 
        <path> {image_paths} </path>
        <id> {id} </id>
        </image>
"""

def llm_summary_image(_element_content):
    image_urls: list[str] = []
    system_prompts: list[str] = []
    user_prompts: list[str] = []
    for idx, item in enumerate(_element_content):
        previous_context = _element_content[idx - 1]["content_text"] if idx - 1 >= 0 else None
        next_context = _element_content[idx + 1]["content_text"] if idx + 1 < len(_element_content) else None
        id_str = item["metadata"]["id"]
        html_str = item["content_text"]
        soup = BeautifulSoup(html_str, "html.parser")
        img_tag = soup.find("img")
        # equation_tag = soup.find(attrs={'data-category': 'equation'})
        if img_tag:
            src_path = img_tag.get("src")
            image_urls.append(src_path)
            system_prompts.append(get_system_prompt())
            user_prompts.append(
                get_user_image_template(previous_context, next_context, html_str, id_str)
            )
    return image_urls,system_prompts,user_prompts

# print(image_urls)
# for u in user_prompts:
#     print(u + "\n")

In [14]:
image_urls,system_prompts,user_prompts = llm_summary_image(_element_content)

In [15]:
from langchain_teddynote.models import MultiModal

llm = get_gpt()
multimodal_llm = MultiModal(llm)
answer = multimodal_llm.batch(
    image_urls, system_prompts, user_prompts, display_image=False
)
# print(answer)

In [16]:
answer

['<image>\n<title>Sequence-to-Sequence Model with LSTM Reading Input in Reverse</title>\n<summary>\nThis image shows how a special type of neural network called an LSTM processes an input sentence and generates an output sentence. The input sentence is "ABC," and the model reads it in reverse order (starting from C, then B, then A). After reading the input, the model produces the output sentence "WXYZ" step by step. It stops generating words when it outputs a special end-of-sentence token labeled "<EOS>." Reading the input backward helps the model learn better by making the connections between words easier to understand.\n</summary>\n<entities>\n- LSTM (Long Short-Term Memory) neural network\n- Input sentence: "ABC"\n- Output sentence: "WXYZ"\n- End-of-sentence token: "<EOS>"\n- Reverse reading of input sequence\n</entities>\n<path> <img src=".cache\\files\\seq2seq\\image_12.png"/> </path>\n<id> 12 </id>\n</image>',
 '<image>\n<title>Visualization of Relationship Sentences in a Two-Dim

In [17]:
# arr = ['<image>\n<title>도표 8. DDR4 가격 추이</title>\n<summary>이 그래프는 2021년 1월부터 2025년 8월까지 DDR4 메모리 가격의 변동 추이를 보여준다. 그래프에는 현물 가격과 고정 가격 두 가지가 표시되어 있으며, 초기에는 가격이 약 5달러에서 시작해 점차 하락하여 2023년 중반에는 1.5달러 근처까지 떨어졌다. 이후 가격은 다시 상승세로 돌아서 2025년 8월에는 약 6달러에 도달하는 모습을 나타낸다.</summary>\n<entities>\n- DDR4 메모리\n- 현물 가격\n- 고정 가격\n- 가격 변동 추이\n- 기간: 2021년 1월 ~ 2025년 8월\n- 단위: 달러 ($)\n</entities>\n<path> <img src="./test/hana/image_43.png"/> </path>\n<id> 43 </id>\n</image>', '<image>\n<title>도표 9. DDR5 가격 추이</title>\n<summary>이 그래프는 2023년 1월부터 7월까지 DDR5 메모리 가격의 변동 추이를 보여준다. 두 가지 가격이 표시되어 있는데, 하나는 현물 가격(진한 녹색 선)이고 다른 하나는 고정 가격(회색 선)이다. 초기에는 가격이 약 5.5달러에서 시작해 3.8달러까지 하락했다가 이후 점차 상승하여 7달러 이상으로 증가하는 추세를 보인다. 고정 가격은 현물 가격보다 낮거나 비슷한 수준에서 움직이다가 후반부에 상승하는 경향을 나타낸다.</summary>\n<entities>DDR5, 가격 추이, 현물 가격, 고정 가격, 2023년 1월~7월, 달러($)</entities>\n<path> <img src="./test/hana/image_46.png"/> </path>\n<id>46</id>\n</image>', '<image>\n<title>도표 11. SLC 8Gb 가격 추이 (SLC 8Gb Price Trend)</title>\n<summary>\nThis line graph shows the price trend of SLC 8Gb memory chips over time, from January 2021 (21.1) to August 2025 (25.8). The vertical axis represents the price in dollars ($), ranging from 0 to 12. Two price lines are depicted: the 현물 가격 (spot price) in blue-green and the 고정 가격 (fixed price) in gray. The spot price starts around $5.5 in early 2021, rises to above $8 by early 2022, then declines and stabilizes around $6.5 until early 2024. After that, it sharply increases to nearly $10 by mid-2025 before slightly dropping. The fixed price remains relatively stable around $4.5 until late 2024, then drops sharply to below $2.5 before gradually rising again to about $3.5 by mid-2025.\n</summary>\n<entities>\n- SLC 8Gb memory chip\n- Price in USD ($)\n- 현물 가격 (Spot Price)\n- 고정 가격 (Fixed Price)\n- Time period: January 2021 to August 2025\n- Data sources: DRAMExchange, 하나증권 (Hana Securities)\n</entities>\n<path> <img src="./test/hana/image_50.png"/> </path>\n<id> 50 </id>\n</image>', '<image>\n<title>현물 가격과 고정 가격 추이 (2021년 1월 ~ 2025년 8월 예상)</title>\n<summary>이 그래프는 2021년 1월부터 2025년 8월까지의 현물 가격과 고정 가격 변동 추이를 보여준다. 현물 가격은 2022년 초에 약 4달러까지 상승했다가 이후 점차 하락하여 2024년 초까지 2.7달러 근처에서 유지되다가 2025년부터 다시 상승하는 경향을 보인다. 고정 가격은 현물 가격보다 전반적으로 낮거나 비슷한 수준을 유지하며, 2024년 중반 이후부터는 현물 가격보다 낮은 수준에서 완만하게 상승하는 모습을 나타낸다.</summary>\n<entities>\n- 현물 가격 (Spot Price)\n- 고정 가격 (Fixed Price)\n- 기간: 2021년 1월 ~ 2025년 8월 (예상 포함)\n- 단위: 달러($)\n- 출처: DRAMExchange, 하나증권\n</entities>\n<path> <img src="./test/hana/image_52.png"/> </path>\n<id> 52 </id>\n</image>', '<image>\n<title>도표 12. 주요 DRAM 업체들의 연초 이후 주가 추이</title>\n<summary>이 그래프는 2024년 12월 31일을 기준으로 주요 DRAM 업체들인 삼성전자, SK하이닉스, Micron, Nanya의 주가 변동 추이를 나타내고 있다. 각 업체의 주가는 2024년 1월부터 9월까지의 기간 동안 변동을 보였으며, 특히 Nanya의 주가가 가장 큰 폭으로 상승하여 210 이상을 기록한 반면, Micron은 상대적으로 안정적인 흐름을 보이며 100 근처에서 움직였다. 삼성전자와 SK하이닉스는 중간 수준의 상승세를 유지하였다.</summary>\n<entities>\n- 삼성전자 (Samsung Electronics)\n- SK하이닉스 (SK Hynix)\n- Micron\n- Nanya\n- 주가 추이 (Stock Price Trend)\n- 기간: 2024년 1월 ~ 9월\n</entities>\n<path> <img src="./test/hana/image_57.png"/> </path>\n<id> 57 </id>\n</image>']

# html_str = data[0]  # 배열에서 첫번째 문자열 추출

# input_json_path = f"{output_file_basename}_merged_1.json"  # JSON 파일 경로
# with open(input_json_path, "r", encoding="utf-8") as f:
#     data = json.load(f)  # JSON 배열 데이터 로드

def change_image_text_to_summary(answer,_element_content):
    for html_str in answer:
        soup = BeautifulSoup(html_str, "html.parser")
        id_tag = soup.find("id")  # id 태그 찾기
        if id_tag:
            id_value = id_tag.text.strip()  # 텍스트 추출 후 공백 제거
            # data[id_value]['content_text'] = html_str
            _element_content[int(id_value)]["content_text"] = html_str
            print("id 값:", id_value)
        else:
            print("id 태그를 찾을 수 없습니다.")
    return _element_content

# with open(f"{output_file_basename}_merged_2.json", "w", encoding="utf-8") as json_file:
#     json.dump(
#         data,
#         json_file,
#     )

In [18]:
change_element_content = change_image_text_to_summary(answer,_element_content)

id 값: 12
id 값: 68
id 값: 69
id 값: 75


### 방정식 설명 추가 로직

In [20]:
import json
import base64
import os
from bs4 import BeautifulSoup


def get_system_prompt():
    return """You are an expert in extracting useful information from IMAGE.
    In particular, you specialize in analyzing papers.
    With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval."""


def get_user_equation_template(previous_context, next_context, equation, id):
    return f"""
        You are an AI assistant who analyzes formulas written in LaTeX format in the paper.
        Explain the given equation easily with an appropriate example so that even a 5-year-old can understand it.
        Explain the equation so that it can be understood without the equation by sufficiently solving it.

        [Preceding paragraph text]
        {previous_context}
        
        [Equation]
        {equation}

        [Following paragraph text]
        {next_context}

        
        # Output Format:
        <title>
        <explain>
        <examples> 
        <id>{id}</id>
"""



# # 저장 폴더가 없으면 생성
# input_json_path = f"{output_file_basename}_merged_2.json"  # JSON 파일 경로
# # print(input_json_path)
# with open(input_json_path, "r", encoding="utf-8") as f:
#     data = json.load(f)  # JSON 배열 데이터 로드

def change_equation(change_element_content):
    system_prompts: list[str] = []
    user_prompts: list[str] = []

    for idx, item in enumerate(change_element_content):
        previous_context = change_element_content[idx - 1]["content_text"] if idx - 1 >= 0 else None
        next_context = change_element_content[idx + 1]["content_text"] if idx + 1 < len(change_element_content) else None
        id_str = item["metadata"]["id"]
        html_str = item["content_text"]
        soup = BeautifulSoup(html_str, "html.parser")
        equation_tag = soup.find(attrs={"data-category": "equation"})
        if equation_tag:
            system_prompts.append(get_system_prompt())
            user_prompts.append(
                get_user_equation_template(
                    previous_context, next_context, equation_tag.get_text(), id_str
                )
            )
    return system_prompts,user_prompts
# for u in user_prompts:
#     print(u + "\n")

In [21]:
system_prompts,user_prompts = change_equation(change_element_content)

In [22]:
def create_messages(system_prompt, user_prompt):
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_prompt,
                },
            ],
        },
    ]
    return messages

In [23]:
def request_llm_equation(system_prompts, user_prompts):
    messages = []
    for system_prompt, user_prompt in zip(system_prompts, user_prompts):
        message = create_messages(system_prompt, user_prompt)
        messages.append(message)
    llm = get_gpt()
    return llm.batch(messages)

answer = request_llm_equation(system_prompts, user_prompts)

In [24]:
answer

[AIMessage(content='<title>Understanding the Basic Recurrent Neural Network (RNN) Equation</title>\n<explain>\nThis equation describes how a Recurrent Neural Network (RNN) processes a sequence of inputs step-by-step to produce outputs. Imagine you have a list of things coming in one after another, like words in a sentence or notes in a song. At each step (time t), the RNN looks at the current input (xt) and also remembers what it learned from the previous step (ht-1). It combines these two pieces of information using some weights (which are like knobs that control how much each part matters) and then applies a special function called "sigmoid" that squashes the result into a number between 0 and 1. This new number (ht) is the RNN\'s updated memory or understanding at step t. Finally, it uses this memory to produce an output (yt) by multiplying it with another set of weights. This way, the RNN can keep track of information over time and make predictions or decisions based on the whole s

In [ ]:
# for r in response:
#     print(r)
    # print(r.content +'\n')

In [25]:
# input_json_path = f"{output_file_basename}_merged_2.json"  # JSON 파일 경로
# with open(input_json_path, "r", encoding="utf-8") as f:
#     data = json.load(f)  # JSON 배열 데이터 로드

def add_eqaution_description(answer,change_element_content):
    for r in answer:
        html_str = r.content
        print(r)
        soup = BeautifulSoup(html_str, "html.parser")
        id_tag = soup.find("id")  # id 태그 찾기
        if id_tag:
            id_value = id_tag.text.strip()  # 텍스트 추출 후 공백 제거
            new_p = soup.new_tag("p")
            new_p.string = html_str
            change_element_content[int(id_value)]["content_text"] = (
                change_element_content[int(id_value)]["content_text"] + html_str
            )
            print("id 값:", id_value)
        else:
            print("id 태그를 찾을 수 없습니다.")
    return change_element_content


# with open(
#     f"{output_file_basename}_merged_final.json", "w", encoding="utf-8"
# ) as json_file:
#     json.dump(
#         data,
#         json_file,
#     )

In [26]:
final_content = add_eqaution_description(answer,change_element_content)

content='<title>Understanding the Basic Recurrent Neural Network (RNN) Equation</title>\n<explain>\nThis equation describes how a Recurrent Neural Network (RNN) processes a sequence of inputs step-by-step to produce outputs. Imagine you have a list of things coming in one after another, like words in a sentence or notes in a song. At each step (time t), the RNN looks at the current input (xt) and also remembers what it learned from the previous step (ht-1). It combines these two pieces of information using some weights (which are like knobs that control how much each part matters) and then applies a special function called "sigmoid" that squashes the result into a number between 0 and 1. This new number (ht) is the RNN\'s updated memory or understanding at step t. Finally, it uses this memory to produce an output (yt) by multiplying it with another set of weights. This way, the RNN can keep track of information over time and make predictions or decisions based on the whole sequence it 

In [ ]:
# with open(f"{output_file_basename}_merged_final.json", "r", encoding="utf-8") as f:
#     data = json.load(f)  # JSON 배열 데이터 로드

In [27]:
def get_prompt_user_request() -> ChatPromptTemplate:
    prompt = """
    You are an expert academic explainer tasked with simplifying difficult and complex research papers. Your responses must be:

    - **Professional and detailed**: Use precise technical terms but always explain them thoroughly.
    - **Clear and accessible**: Include sufficient explanations and relevant examples so that even a young child can understand.
    - **Formatted in Markdown**: Use headings, lists, and emphasis where appropriate for readability.
    - **Source-aware**: If the original document references specific pages or sections, always include the source page number or reference.
    - **Image-inclusive**: If the input contains image or figure tags, display the image path or URL alongside your explanation.


    When given a text input from a paper, first break down complex concepts step-by-step, 
    illustrate with examples or analogies, and clearly indicate the source pages if applicable. 
    Once text is entered in the paper, we first break down complex concepts step by step,
    Explain with examples or parables and, if applicable, clearly display the source page.
    Provides a final cohesive summary in Markdown format.
    To represent the equation (x_1, \\cdots, x_T), switch to $(x_1, \\cdots, x_T)$ sentence and output it
    If <img src='example'/> exists in the document you refer to, output the src path as it is
    

    ---

    **Example Usage:**

    Input: "Explain the LSTM model from page 5 that includes 4 layers and GPU parallelization. Include any figures if present."

    Output: *(Detailed, clear Markdown explanation with examples, source page 5 cited, and image paths if any).* 
    ---

    Input: "Please explain the key mechanism equation
"
    Output: $(x_1, \\cdots, x_T)$
    ---

    **Important:**
    Make sure to answer in Korean except for the image path and equation
 
    
    ** User Request:**
    {question}

    ** context :**
    {context}    
"""
    return ChatPromptTemplate.from_template(prompt)

In [28]:
user_input = "이논문이 해결하고자하는 문제가머야 ?"
docs = []
for d in final_content:
    doc = Document(page_content=d["content_text"], metadata=d["metadata"])
    docs.append(doc)

faiss_retriever = get_retriever(docs)
bm25_retriever = get_bm25_retriever(docs)
esenmble_retriever = get_esenmble_retriever(faiss_retriever, bm25_retriever)
rerank = get_reranker(esenmble_retriever, user_input)
reorder_context = reorder_documents(rerank)
prompt = get_prompt_user_request()
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": user_input, "context": reorder_context})
print(response)

# 이 논문이 해결하고자 하는 문제

이 논문은 **LSTM(Long Short-Term Memory)** 모델을 사용하여 **입력 시퀀스**에 대응하는 **출력 시퀀스의 조건부 확률**을 추정하는 문제를 다루고 있습니다. 

---

## 문제 설명

- 입력 시퀀스 $(x_1, \cdots, x_T)$가 주어졌을 때, 이에 대응하는 출력 시퀀스 $(y_1, \cdots, y_{T'})$의 확률 $p(y_1, \cdots, y_{T'} | x_1, \cdots, x_T)$를 계산하는 것이 목표입니다.  
- 여기서 입력 시퀀스와 출력 시퀀스의 길이 $T$와 $T'$는 서로 다를 수 있습니다.  
- LSTM은 입력 시퀀스를 고정된 차원의 벡터 표현 $v$로 변환한 뒤, 이 벡터를 초기 상태로 하여 출력 시퀀스의 확률을 계산합니다.  

---

## 왜 이 문제가 중요한가?

- 시퀀스 데이터(예: 문장, 음성, 동작 등)는 길이가 다양하고, 각 시퀀스의 각 요소가 서로 복잡하게 연결되어 있습니다.  
- 전통적인 모델들은 긴 시퀀스에서 앞부분과 뒷부분 사이의 관계를 잘 포착하지 못하는 문제가 있습니다.  
- LSTM은 이런 긴 시퀀스 내의 **장기 의존성(long-term dependencies)** 문제를 해결할 수 있는 구조로, 입력과 출력 시퀀스 간의 복잡한 관계를 모델링하는 데 적합합니다.  

---

## 쉽게 이해하기 위한 비유

- **입력 시퀀스**를 여러 문장으로 이루어진 이야기라고 생각해보세요.  
- LSTM은 이 이야기를 읽고, 이야기의 핵심 내용을 요약한 메모(벡터 $v$)를 만듭니다.  
- 그리고 이 메모를 바탕으로 새로운 이야기를 한 문장씩 차례대로 만들어 나갑니다.  
- 각 문장은 이전에 만든 문장들과 이야기의 핵심 메모를 참고해서 만들어집니다.  

---

## 핵심 수식

논문에서 제시한 조건부 확률은 다음과 같습니다 (3페이지):

$$
p(y_{1}, \cdots, y_{T^{\prime}} | x_{1}, \c

# 이 논문이 해결하고자 하는 문제

이 논문은 **LSTM(Long Short-Term Memory)** 모델을 사용하여 **입력 시퀀스** $(x_1, \cdots, x_T)$에 대해 길이가 다를 수 있는 **출력 시퀀스** $(y_1, \cdots, y_{T'})$의 조건부 확률 $p(y_1, \cdots, y_{T'} | x_1, \cdots, x_T)$를 추정하는 문제를 다룹니다. 

---

## 문제의 핵심

- **입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있음**  
  예를 들어, 영어 문장(입력)이 10단어이고, 그에 대응하는 프랑스어 문장(출력)이 12단어일 수 있습니다.  
- **입력 시퀀스를 고정된 차원의 벡터로 요약하는 것**  
  LSTM은 입력 시퀀스 전체를 읽고, 마지막 숨겨진 상태(hidden state)를 통해 이 시퀀스를 하나의 벡터 $v$로 요약합니다.  
- **요약된 벡터 $v$를 바탕으로 출력 시퀀스를 생성**  
  이 벡터 $v$를 초기 상태로 하여, LSTM 언어 모델(LM)이 출력 시퀀스의 각 단어를 순차적으로 예측합니다.

---

## 문제를 쉽게 이해하기 위한 비유

- **입력 시퀀스**: 여러 장의 그림이 순서대로 놓여 있는 책  
- **출력 시퀀스**: 그 그림들을 보고 새롭게 이야기를 만드는 것  
- LSTM은 먼저 책 전체를 쭉 읽고(입력 시퀀스 요약), 그 내용을 머릿속에 저장합니다(벡터 $v$).  
- 그 다음, 머릿속에 저장한 내용을 바탕으로 한 문장씩 차례대로 이야기를 만들어 나갑니다(출력 시퀀스 생성).

---

## 수식으로 표현된 문제 (출처: 3페이지)

$$
p(y_{1}, \cdots, y_{T'} | x_{1}, \cdots, x_{T}) = \prod_{t=1}^{T'} p(y_t | v, y_1, \cdots, y_{t-1})
$$

- 이 식은 출력 시퀀스의 각 단어 $y_t$가,  
  - 입력 시퀀스 전체를 요약한 벡터 $v$와  
  - 지금까지 생성된 출력 단어들 $y_1, \cdots, y_{t-1}$에 조건부로 의존한다는 의미입니다.  
- 즉, LSTM은 이전에 생성한 단어들과 입력 시퀀스 요약 정보를 모두 고려하여 다음 단어를 예측합니다.

---

## 요약

| 문제점 | 설명 |
|--------|-------|
| 입력과 출력 시퀀스 길이 불일치 | 입력과 출력의 길이가 다를 수 있어, 단순한 매핑이 어려움 |
| 시퀀스 전체를 고정된 크기로 요약 | LSTM의 마지막 숨겨진 상태를 사용해 입력 시퀀스를 하나의 벡터로 요약 |
| 조건부 확률 추정 | 요약 벡터와 이전 출력 단어들을 바탕으로 다음 단어를 예측하는 확률 모델 구축 |

이 논문은 위 문제를 해결하기 위해 LSTM 기반의 시퀀스-투-시퀀스(sequence-to-sequence) 모델을 제안하고, 이를 통해 자연어 번역 등 다양한 시퀀스 변환 문제에 적용하고자 합니다.

---

# 참고  
- 문제 정의 및 수식: **3페이지**  
- 수식 설명 및 예시: 3페이지 (id: 26)  

---

필요하면 추가 설명이나 다른 부분도 알려주세요!

# 이 논문이 해결하고자 하는 문제

이 논문은 **LSTM(Long Short-Term Memory)** 모델을 사용하여 **입력 시퀀스**에 대응하는 **출력 시퀀스의 조건부 확률**을 추정하는 문제를 다루고 있습니다. 

---

## 문제 설명

- 입력 시퀀스 $(x_1, \cdots, x_T)$가 주어졌을 때, 이에 대응하는 출력 시퀀스 $(y_1, \cdots, y_{T'})$의 확률 $p(y_1, \cdots, y_{T'} | x_1, \cdots, x_T)$를 계산하는 것이 목표입니다.  
- 여기서 입력 시퀀스와 출력 시퀀스의 길이 $T$와 $T'$는 서로 다를 수 있습니다.  
- LSTM은 입력 시퀀스를 고정된 차원의 벡터 표현 $v$로 변환한 뒤, 이 벡터를 초기 상태로 하여 출력 시퀀스의 확률을 계산합니다.  

---

## 왜 이 문제가 중요한가?

- 시퀀스 데이터(예: 문장, 음성, 동작 등)는 길이가 다양하고, 각 시퀀스의 각 요소가 서로 복잡하게 연결되어 있습니다.  
- 전통적인 모델들은 긴 시퀀스에서 앞부분과 뒷부분 사이의 관계를 잘 포착하지 못하는 문제가 있습니다.  
- LSTM은 이런 긴 시퀀스 내의 **장기 의존성(long-term dependencies)** 문제를 해결할 수 있는 구조로, 입력과 출력 시퀀스 간의 복잡한 관계를 모델링하는 데 적합합니다.  

---

## 쉽게 이해하기 위한 비유

- **입력 시퀀스**를 여러 문장으로 이루어진 이야기라고 생각해보세요.  
- LSTM은 이 이야기를 읽고, 이야기의 핵심 내용을 요약한 메모(벡터 $v$)를 만듭니다.  
- 그리고 이 메모를 바탕으로 새로운 이야기를 한 문장씩 차례대로 만들어 나갑니다.  
- 각 문장은 이전에 만든 문장들과 이야기의 핵심 메모를 참고해서 만들어집니다.  

---

## 핵심 수식

논문에서 제시한 조건부 확률은 다음과 같습니다 (3페이지):

$$
p(y_{1}, \cdots, y_{T^{\prime}} | x_{1}, \cdots, x_{T}) = \prod_{t=1}^{T^{\prime}} p(y_t | v, y_1, \cdots, y_{t-1})
$$

- 즉, 출력 시퀀스의 각 단어 $y_t$는 입력 시퀀스의 요약 $v$와 이전에 생성된 단어들 $y_1, \cdots, y_{t-1}$에 조건부로 의존합니다.  
- 이 수식은 LSTM이 어떻게 순차적으로 출력을 생성하는지 보여줍니다.  

---

## 요약

| 항목 | 설명 |
|---|---|
| **해결하고자 하는 문제** | 입력 시퀀스에 대해 대응하는 출력 시퀀스의 조건부 확률을 정확히 추정하는 것 |
| **주요 도전 과제** | 입력과 출력 시퀀스 길이가 다르고, 긴 시퀀스 내 장기 의존성을 학습하는 어려움 |
| **해결 방법** | LSTM을 사용해 입력 시퀀스를 고정 차원 벡터로 요약하고, 이를 초기 상태로 출력 시퀀스를 생성 |
| **적용 분야 예시** | 기계 번역, 음성 인식, 텍스트 생성 등 시퀀스-투-시퀀스 문제 |

---

## 참고 문헌 및 페이지

- 문제 정의 및 수식: 3페이지 (id: 25, 26)  
- 관련 설명: 3페이지  
- 추가 설명 및 현상 분석: 4페이지 (id: 46)  

---

필요하면 더 구체적인 부분이나 다른 섹션도 설명해 드릴 수 있습니다!